# Apache Scratch Pad


This notebook will aim to query the Augur DB to access the neccessary information to be able to get the following issue metrics dirived from the GitHub Community Metrics working document https://docs.google.com/document/d/1Yocr6fk0J8EsVZnJwoIl3kRQaI94tI-XHe7VSMFT0yM/edit?usp=sharing

Any necessary computations from the data to get the metric value will be done as the queries are determined

In [2]:
import psycopg2
import pandas as pd 
import sqlalchemy as salc
import json
import os
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (15, 5)
import warnings
warnings.filterwarnings('ignore')

with open("../../comm_cage.json") as config_file:
    config = json.load(config_file)

In [3]:
database_connection_string = 'postgresql+psycopg2://{}:{}@{}:{}/{}'.format(config['user'], config['password'], config['host'], config['port'], config['database'])

dbschema='augur_data'
engine = salc.create_engine(
    database_connection_string,
    connect_args={'options': '-csearch_path={}'.format(dbschema)})

### Query Connection Example

In [4]:
#add your repo name(s) here of the repo(s) you want to query if known (and in the database)
repo_name_set = ['augur']
repo_set = []
repo_ids = []

repolist = pd.DataFrame() 

for repo_name in repo_name_set:
    repo_query = salc.sql.text(f"""
                 SET SCHEMA 'augur_data';
                 SELECT 
                    b.repo_id
                FROM
                    repo_groups a,
                    repo b
                WHERE
                    a.repo_group_id = b.repo_group_id AND
                    b.repo_name = \'{repo_name}\'
        """)
    print(repo_query)

    repolist = pd.read_sql(repo_query, con=engine);
    d, a = {}, []
    for repo in repolist:
        # rowproxy.items() returns an array like [(key0, value0), (key1, value1)]
        for column, value in repolist.items():
            # build up the dictionary
            d = {**d, **{column: value}}
        #$repo_set.append(d)
        repo_ids = repolist['repo_id']

    
        
    #repo_id =t.mappings().all()[0].get('repo_id')
    repo_set.append(repolist['repo_id'][0])

# repo_set = repolist['repo_id'][0]
print(repo_set)
print(repolist['repo_id'][0])



                 SET SCHEMA 'augur_data';
                 SELECT 
                    b.repo_id
                FROM
                    repo_groups a,
                    repo b
                WHERE
                    a.repo_group_id = b.repo_group_id AND
                    b.repo_name = 'augur'
        
[36113]
36113


In [5]:
print(repo_set)

[36113]


In [6]:
#Take this out of quotes if you want to manually assign a repo_id number(s)
#repo_set = [25440]

## Example Query for Issue Analysis

In [7]:
df_issues = pd.DataFrame()

for repo_id in repo_set: 

    pr_query = salc.sql.text(f"""
                SELECT
                    r.repo_name,
					i.issue_id AS issue, 
					i.gh_issue_number AS issue_number,
					i.gh_issue_id AS gh_issue,
					i.created_at AS created, 
					i.closed_at AS closed
                FROM
                	repo r,
                    issues i
                WHERE
                	r.repo_id = i.repo_id AND
                    i.repo_id = \'{repo_id}\'
        """)
    df_current_repo = pd.read_sql(pr_query, con=engine)
    df_issues = pd.concat([df_issues, df_current_repo])

df_issues = df_issues.reset_index()
df_issues.drop("index", axis=1, inplace=True)
        
df_issues.head()

,repo_name,issue,issue_number,gh_issue,created,closed
0,augur,91116,1525,1063293122,2021-11-25 08:33:28,NaT
1,augur,91121,1518,1055344486,2021-11-16 21:21:05,NaT
2,augur,91125,1514,1052511565,2021-11-13 01:32:55,NaT
3,augur,91128,1510,1041280595,2021-11-01 15:09:06,NaT
4,augur,91136,1495,1027800478,2021-10-15 20:55:07,NaT


# Apache Metadata Queries

In [8]:
repository_list = pd.DataFrame()
pull_requests = pd.DataFrame()
commits = pd.DataFrame()
issues = pd.DataFrame()
contributors = pd.DataFrame()

repository_list_SQL = salc.sql.text(f"""
        select repo.repo_id, repo_git from repo
        where repo_git like '%apache%';
    """)

pull_requests_SQL = salc.sql.text(f"""
        select repo.repo_id, repo_git, count(*) from repo, pull_requests 
        where repo_git like '%apache%'
        and 
        repo.repo_id = pull_requests.repo_id
        group by repo.repo_id, repo_git; 
    """)

commits_SQL = salc.sql.text(f"""
        select repo.repo_id, repo_git, count(*) from repo, commits 
        where repo_git like '%apache%'
        and 
        repo.repo_id = commits.repo_id
        group by repo.repo_id, repo_git; 
    """)

issues_SQL = salc.sql.text(f"""
        select repo.repo_id, repo_git, count(*) from repo, issues 
        where repo_git like '%apache%'
        and 
        repo.repo_id = issues.repo_id
        group by repo.repo_id, repo_git; 
            """)

contributors_SQL = salc.sql.text(f"""
        select distinct * from (
        select distinct a.cntrb_id as cntrb_id, a.alias_email as email FROM
        (
        select contributors.cntrb_id, alias_email from contributors, contributors_aliases where (contributors.cntrb_canonical like '%redhat.com%' or 
        contributors_aliases.alias_email like '%redhat.com%') 
        and contributors.cntrb_id = contributors_aliases.cntrb_id
        ) a
        UNION
        select cntrb_id as cntrb_id, cntrb_canonical as email 
        from contributors where cntrb_canonical like '%redhat.com%' ) b;
    """)


repository_list = pd.read_sql(repository_list_SQL, con=engine)
pull_requests = pd.read_sql(pull_requests_SQL, con=engine)
commits = pd.read_sql(commits_SQL, con=engine)
issues = pd.read_sql(issues_SQL, con=engine)
contributors = pd.read_sql(contributors_SQL, con=engine)


In [9]:
print(repository_list)

      repo_id                                           repo_git
0       30260    https://github.com/apache/turbine-fulcrum-build
1       29994               https://github.com/apache/unomi-site
2       28779    https://github.com/apache/couchdb-erlang-bcrypt
3       28765  https://github.com/apache/fineract-cn-stellar-...
4       29974                 https://github.com/apache/dubbo-js
...       ...                                                ...
2265    30319  https://github.com/apache/sling-org-apache-sli...
2266    29345                 https://github.com/apache/ws-woden
2267    29669     https://github.com/apache/fineract-cn-template
2268    30525               https://github.com/apache/cloudstack
2269    30347     https://github.com/apache/incubator-mxnet-site

[2270 rows x 2 columns]


In [10]:
print(pull_requests)

      repo_id                                           repo_git  count
0       28445          https://github.com/apache/ofbiz-framework    441
1       28447       https://github.com/apache/incubator-hivemall    238
2       28449  https://github.com/apache/incubator-flagon-use...    140
3       28451             https://github.com/apache/fluo-website    177
4       28453           https://github.com/apache/couchdb-erlfdb     37
...       ...                                                ...    ...
1345    30706                     https://github.com/apache/solr    419
1346    30707             https://github.com/apache/commons-bcel    108
1347    30708       https://github.com/apache/myfaces-master-pom     34
1348    30710        https://github.com/apache/geronimo-jwt-auth      4
1349    30711       https://github.com/apache/calcite-avatica-go     54

[1350 rows x 3 columns]


In [11]:
print(commits)

    repo_id                                           repo_git  count
0     28498   https://github.com/apache/maven-changelog-plugin    620
1     28515      https://github.com/apache/servicecomb-samples   1584
2     28536  https://github.com/apache/manifoldcf-integrati...    159
3     28548          https://github.com/apache/incubator-doris  43866
4     28601                 https://github.com/apache/sandesha   5219
..      ...                                                ...    ...
77    30555                https://github.com/apache/zookeeper  17836
78    30619  https://github.com/apache/sling-org-apache-sli...    308
79    30647  https://github.com/apache/sling-org-apache-sli...     49
80    30666               https://github.com/apache/maven-site   7558
81    30693                 https://github.com/apache/libcloud  24308

[82 rows x 3 columns]


In [12]:
print(issues)

      repo_id                                           repo_git  count
0       28445          https://github.com/apache/ofbiz-framework    438
1       28447       https://github.com/apache/incubator-hivemall    245
2       28449  https://github.com/apache/incubator-flagon-use...    203
3       28450              https://github.com/apache/ode-console      1
4       28451             https://github.com/apache/fluo-website    213
...       ...                                                ...    ...
1683    30707             https://github.com/apache/commons-bcel    113
1684    30708       https://github.com/apache/myfaces-master-pom     38
1685    30709                    https://github.com/apache/derby      7
1686    30710        https://github.com/apache/geronimo-jwt-auth      6
1687    30711       https://github.com/apache/calcite-avatica-go     55

[1688 rows x 3 columns]


In [13]:
print(contributors)

      cntrb_id                         email
0        25433       hema.veeradhi@gmail.com
1        25433           hveeradh@redhat.com
2        25434             kachau@redhat.com
3        25435  mcliffor@mcliffor.remote.csb
4        25435           mcliffor@redhat.com
...        ...                           ...
2542   2424541            prsurve@redhat.com
2543   2425751               vdas@redhat.com
2544   2425777             harold@redhat.com
2545   2426346              yrabl@redhat.com
2546   2451295              tgeer@redhat.com

[2547 rows x 2 columns]


## Red Hat Analysis Notes: 43 Repository Sample
1. How many unique RH'ers are contributing to the represented apache projects. Output the unique cntrb_ids for each RH'er. Count them. Look at total contributors. 
2. Percentage of commits (possibly LOC) from RH'ers.
3. Percentage of contributors.
4. Percentage of LOC.
5. Illustrate SBOM's
6. Illustrate Dependencies


In [14]:
rh_commits = pd.DataFrame()
repository_list = pd.DataFrame()

repository_list_SQL = salc.sql.text(f"""
        select repo.repo_id, repo_git from repo
        where repo_git like '%apache%';
    """)

rh_commits_SQL = salc.sql.text(f"""
        SELECT
            * 
        FROM
            (
            SELECT
                * 
            FROM
                (
                    ( SELECT * FROM repo WHERE repo_git LIKE'%apache%' ) repo
                    LEFT JOIN (
                    SELECT
                        RH.*,
                        committer.*,
                        ( RH.RH_commits :: DECIMAL / ( committer.all_commits + 1 ) :: DECIMAL ) :: DECIMAL AS commit_pct,
                        ( RH.RH_added :: DECIMAL / ( committer.all_added + 1 ) :: DECIMAL ) :: DECIMAL AS added_pct,
                        ( RH.RH_deleted :: DECIMAL / ( committer.all_deleted + 1 ) :: DECIMAL ) :: DECIMAL AS deleted_pct,
                        ( RH.RH_whitespace :: DECIMAL / ( committer.all_whitespace + 1 ) :: DECIMAL ) :: DECIMAL AS whitespace_pct 
                    FROM
                        (
                        SELECT
                            b.repo_id AS all_repos_id,
                            COUNT ( C.cmt_id ) AS all_commits,
                            SUM ( C.cmt_added ) AS all_added,
                            SUM ( C.cmt_removed ) AS all_deleted,
                            SUM ( C.cmt_whitespace ) AS all_whitespace 
                        FROM
                            repo_groups A,
                            repo b,
                            commits C 
                        WHERE
                            A.repo_group_id = b.repo_group_id 
                            AND b.repo_id = C.repo_id 
                            AND C.cmt_id IS NOT NULL 
                            AND C.cmt_added IS NOT NULL 
                            AND C.cmt_removed IS NOT NULL 
                            AND C.cmt_whitespace IS NOT NULL 
                            AND b.repo_git LIKE'%apache%' 
                        GROUP BY
                            A.repo_group_id,
                            b.repo_id 
                        ORDER BY
                            A.repo_group_id,
                            b.repo_id 
                        ) committer
                        LEFT JOIN (
                        SELECT
                            b.repo_id AS RH_repos_id,
                            COUNT ( C.cmt_id ) AS RH_commits,
                            SUM ( C.cmt_added ) AS RH_added,
                            SUM ( C.cmt_removed ) AS RH_deleted,
                            SUM ( C.cmt_whitespace ) AS RH_whitespace 
                        FROM
                            repo_groups A,
                            repo b,
                            commits C 
                        WHERE
                            b.repo_git LIKE'%apache%' 
                            AND A.repo_group_id = b.repo_group_id 
                            AND b.repo_id = C.repo_id 
                            AND (
                                C.cmt_author_email IN (
                                SELECT DISTINCT
                                    email 
                                FROM
                                    (
                                    SELECT DISTINCT A
                                        .cntrb_id AS cntrb_id,
                                        A.alias_email AS email 
                                    FROM
                                        (
                                        SELECT
                                            contributors.cntrb_id,
                                            alias_email 
                                        FROM
                                            contributors,
                                            contributors_aliases 
                                        WHERE
                                            ( contributors.cntrb_canonical LIKE'%redhat.com%' OR contributors_aliases.alias_email LIKE'%redhat.com%' ) 
                                            AND contributors.cntrb_id = contributors_aliases.cntrb_id 
                                        ) A UNION
                                    SELECT
                                        cntrb_id AS cntrb_id,
                                        cntrb_canonical AS email 
                                    FROM
                                        contributors 
                                    WHERE
                                        cntrb_canonical LIKE'%redhat.com%' 
                                    ) b 
                                ) or (C.cmt_author_email LIKE'%redhat.com')
                            ) 
                        GROUP BY
                            A.repo_group_id,
                            b.repo_id 
                        ORDER BY
                            A.repo_group_id,
                            b.repo_id 
                        ) RH ON committer.all_repos_id = RH.RH_repos_id 
                    ORDER BY
                        committer.all_repos_id,
                        RH.RH_repos_id 
                    ) allcommits ON repo.repo_id = allcommits.all_repos_id 
                ) pdd 
            ORDER BY
                commit_pct DESC 
            ) b 
        WHERE
            b.all_commits IS NOT NULL
            order by rh_commits nulls last; 

    """)

rh_commits = pd.read_sql(rh_commits_SQL, con=engine)

repository_list = pd.read_sql(repository_list_SQL, con=engine)

print(rh_commits)


    repo_id  repo_group_id                                           repo_git  \
0     30666          25478               https://github.com/apache/maven-site   
1     30555          25478                https://github.com/apache/zookeeper   
2     29753          25478                  https://github.com/apache/ant-ivy   
3     29742          25478                 https://github.com/apache/activemq   
4     30693          25478                 https://github.com/apache/libcloud   
..      ...            ...                                                ...   
77    30285          25478  https://github.com/apache/netbeans-mavenutils-...   
78    28515          25478      https://github.com/apache/servicecomb-samples   
79    28536          25478  https://github.com/apache/manifoldcf-integrati...   
80    28548          25478          https://github.com/apache/incubator-doris   
81    28601          25478                 https://github.com/apache/sandesha   

             repo_path     